In [1]:
import requests
from bs4 import BeautifulSoup
import time
import csv

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import scrapy
import logging
from twisted.internet import reactor

from scrapy.crawler import CrawlerProcess, CrawlerRunner
from scrapy.utils.project import get_project_settings
from multiprocessing import Process, Queue

from urllib.parse import urlencode
from urllib.parse import urlparse
import json
from datetime import datetime
import shadow_useragent

import re

import random 
from collections import OrderedDict
from proxyscrape import create_collector

from fake_headers import Headers
from requests import get

### Collecting Profiles

In [2]:
def grab_profiles(Source_URL):
    
    source = requests.get(Source_URL).text
    domain = "https://scholar.google.co.uk"
    
    proflinks = []

    while True:
        soup = BeautifulSoup(source, "html.parser")
        nxt = soup.find("button", attrs = {"aria-label": "Next"})
        urlWindow = nxt.get_attribute_list("onclick")
        profs = soup.find_all("h3", class_="gs_ai_name")
        for prof in profs:
            prof_cit = domain + str(prof.a.get_attribute_list("href")[0])
            proflinks.append(prof_cit)
        if urlWindow[0] is not None:
            path = Source_URL + urlWindow[0].split("=")[1].replace("\\x26", "&").replace("\\x3d", "=")
            source = requests.get(path).text
        else:
            break
            
    with open("Covent_URLs.csv", mode = "w", newline = "") as Prof_URL:
        Prof_URL_Writer = csv.writer(Prof_URL, delimiter = ",")
        for i in proflinks:
            Prof_URL_Writer.writerow([i])
    
    return proflinks

In [3]:
Author_Profiles = grab_profiles("https://scholar.google.co.uk/citations?view_op=view_org&hl=en&org=9117984065169182779")

### Loading Authors List

In [2]:
with open('Covents_URLs.csv', mode = "r", newline = "") as f:
    reader = csv.reader(f)
    test = []
    for i in reader:
        test.append(i[0])

    for e in test:
        print(e)

https://scholar.google.co.uk/citations?hl=en&user=ywiPqccAAAAJ
https://scholar.google.co.uk/citations?hl=en&user=A93LzQMAAAAJ
https://scholar.google.co.uk/citations?hl=en&user=hZY-wA4AAAAJ
https://scholar.google.co.uk/citations?hl=en&user=tI1RwOIAAAAJ
https://scholar.google.co.uk/citations?hl=en&user=PdZZpBkAAAAJ
https://scholar.google.co.uk/citations?hl=en&user=TfTyMZQAAAAJ
https://scholar.google.co.uk/citations?hl=en&user=GK_0st4AAAAJ
https://scholar.google.co.uk/citations?hl=en&user=LdhLhI0AAAAJ
https://scholar.google.co.uk/citations?hl=en&user=rr94cyYAAAAJ
https://scholar.google.co.uk/citations?hl=en&user=-GsW3HsAAAAJ
https://scholar.google.co.uk/citations?hl=en&user=KTXoxysAAAAJ
https://scholar.google.co.uk/citations?hl=en&user=UB66hN4AAAAJ
https://scholar.google.co.uk/citations?hl=en&user=vnyHhSoAAAAJ
https://scholar.google.co.uk/citations?hl=en&user=RVGRUGgAAAAJ
https://scholar.google.co.uk/citations?hl=en&user=UnaJjYIAAAAJ
https://scholar.google.co.uk/citations?hl=en&user=UA1qV

### BeautifulSoup Paper URL grabber

In [4]:
def grab_profiles(Profile_URL_List):
    count = 0

    Paper_Links = []

    with requests.session() as ses:
        for i in Profile_URL_List:
            start = 0
            while start < 30:
                time.sleep(2.25)
                url = i + f"&cstart={start}00&pagesize=100" 
                s = BeautifulSoup(ses.get(url).text, "html.parser")
                x = s.findAll("a")
                if s.find("button", id = "gsc_bpf_more").has_attr("disabled"):
                    start += 31
                else:
                    start += 1
                for t in x:
                    v = t.get("data-href")
                    if v != None:
                        v = re.sub(r"&cstart=.*&", "&", v)
                        count += 1
                        print(v)
                        Paper_Links.append(v)
        
    for i in range(len(Paper_Links)):
        for j in Paper_Links[i].split("?")[1].split("&"):
            if j.startswith("user"):
                n = j
                Paper_Links[i] = Paper_Links[i].replace("&" + n, "").replace("?", "?"+n+"&")
            
    print(count)
    return Paper_Links

In [ ]:
Paper_Links = grab_profiles(Author_Profiles)

In [ ]:
with open("Paper_URLs.csv", mode = "w", newline = "") as Pap_URL:
    Pap_URL_Writer = csv.writer(Pap_URL, delimiter = ",")
    
    for i in Paper_Links:
        Pap_URL_Writer.writerow([i])

In [2]:
with open('Paper_URLs.csv', mode = "r", newline = "") as f:
    reader = csv.reader(f)
    test = []
    for i in reader:
        test.append(i[0])

    for e in test:
        print(e)

Paper_Links = test

/citations?user=ywiPqccAAAAJ&view_op=view_citation&hl=en&oe=ASCII&pagesize=100&citation_for_view=ywiPqccAAAAJ:wMgC3FpKEyYC
/citations?user=ywiPqccAAAAJ&view_op=view_citation&hl=en&oe=ASCII&pagesize=100&citation_for_view=ywiPqccAAAAJ:PELIpwtuRlgC
/citations?user=ywiPqccAAAAJ&view_op=view_citation&hl=en&oe=ASCII&pagesize=100&citation_for_view=ywiPqccAAAAJ:b1wdh0AR-JQC
/citations?user=ywiPqccAAAAJ&view_op=view_citation&hl=en&oe=ASCII&pagesize=100&citation_for_view=ywiPqccAAAAJ:1yWc8FF-_SYC
/citations?user=ywiPqccAAAAJ&view_op=view_citation&hl=en&oe=ASCII&pagesize=100&citation_for_view=ywiPqccAAAAJ:5awf1xo2G04C
/citations?user=ywiPqccAAAAJ&view_op=view_citation&hl=en&oe=ASCII&pagesize=100&citation_for_view=ywiPqccAAAAJ:5MTHONV0fEkC
/citations?user=ywiPqccAAAAJ&view_op=view_citation&hl=en&oe=ASCII&pagesize=100&citation_for_view=ywiPqccAAAAJ:a0OBvERweLwC
/citations?user=ywiPqccAAAAJ&view_op=view_citation&hl=en&oe=ASCII&pagesize=100&citation_for_view=ywiPqccAAAAJ:9c2xU6iGI7YC
/citations?user=

/citations?user=A93LzQMAAAAJ&view_op=view_citation&hl=en&oe=ASCII&citation_for_view=A93LzQMAAAAJ:0KyAp5RtaNEC
/citations?user=A93LzQMAAAAJ&view_op=view_citation&hl=en&oe=ASCII&citation_for_view=A93LzQMAAAAJ:7T2F9Uy0os0C
/citations?user=A93LzQMAAAAJ&view_op=view_citation&hl=en&oe=ASCII&citation_for_view=A93LzQMAAAAJ:W5xh706n7nkC
/citations?user=A93LzQMAAAAJ&view_op=view_citation&hl=en&oe=ASCII&citation_for_view=A93LzQMAAAAJ:uLbwQdceFCQC
/citations?user=A93LzQMAAAAJ&view_op=view_citation&hl=en&oe=ASCII&citation_for_view=A93LzQMAAAAJ:UHK10RUVsp4C
/citations?user=A93LzQMAAAAJ&view_op=view_citation&hl=en&oe=ASCII&citation_for_view=A93LzQMAAAAJ:ZuybSZzF8UAC
/citations?user=A93LzQMAAAAJ&view_op=view_citation&hl=en&oe=ASCII&citation_for_view=A93LzQMAAAAJ:N5tVd3kTz84C
/citations?user=A93LzQMAAAAJ&view_op=view_citation&hl=en&oe=ASCII&citation_for_view=A93LzQMAAAAJ:VL0QpB8kHFEC
/citations?user=A93LzQMAAAAJ&view_op=view_citation&hl=en&oe=ASCII&citation_for_view=A93LzQMAAAAJ:LjlpjdlvIbIC
/citations

/citations?user=tI1RwOIAAAAJ&view_op=view_citation&hl=en&oe=ASCII&citation_for_view=tI1RwOIAAAAJ:Kaaf24wrr50C
/citations?user=tI1RwOIAAAAJ&view_op=view_citation&hl=en&oe=ASCII&citation_for_view=tI1RwOIAAAAJ:z5I2ou_Dk_MC
/citations?user=tI1RwOIAAAAJ&view_op=view_citation&hl=en&oe=ASCII&citation_for_view=tI1RwOIAAAAJ:Id9pRtCSqO0C
/citations?user=tI1RwOIAAAAJ&view_op=view_citation&hl=en&oe=ASCII&citation_for_view=tI1RwOIAAAAJ:9wUeeRLfbNYC
/citations?user=tI1RwOIAAAAJ&view_op=view_citation&hl=en&oe=ASCII&citation_for_view=tI1RwOIAAAAJ:qKtbcrzMvwAC
/citations?user=tI1RwOIAAAAJ&view_op=view_citation&hl=en&oe=ASCII&citation_for_view=tI1RwOIAAAAJ:vpzZMdrIWDEC
/citations?user=tI1RwOIAAAAJ&view_op=view_citation&hl=en&oe=ASCII&citation_for_view=tI1RwOIAAAAJ:JjPkQosUWiAC
/citations?user=tI1RwOIAAAAJ&view_op=view_citation&hl=en&oe=ASCII&citation_for_view=tI1RwOIAAAAJ:8RJSsxxtMAkC
/citations?user=tI1RwOIAAAAJ&view_op=view_citation&hl=en&oe=ASCII&citation_for_view=tI1RwOIAAAAJ:XyWThvt29VcC
/citations

/citations?user=rr94cyYAAAAJ&view_op=view_citation&hl=en&oe=ASCII&pagesize=100&citation_for_view=rr94cyYAAAAJ:tzM49s52ZIMC
/citations?user=rr94cyYAAAAJ&view_op=view_citation&hl=en&oe=ASCII&pagesize=100&citation_for_view=rr94cyYAAAAJ:NJ774b8OgUMC
/citations?user=rr94cyYAAAAJ&view_op=view_citation&hl=en&oe=ASCII&pagesize=100&citation_for_view=rr94cyYAAAAJ:J_g5lzvAfSwC
/citations?user=rr94cyYAAAAJ&view_op=view_citation&hl=en&oe=ASCII&pagesize=100&citation_for_view=rr94cyYAAAAJ:KlAtU1dfN6UC
/citations?user=rr94cyYAAAAJ&view_op=view_citation&hl=en&oe=ASCII&pagesize=100&citation_for_view=rr94cyYAAAAJ:YFjsv_pBGBYC
/citations?user=rr94cyYAAAAJ&view_op=view_citation&hl=en&oe=ASCII&pagesize=100&citation_for_view=rr94cyYAAAAJ:UebtZRa9Y70C
/citations?user=rr94cyYAAAAJ&view_op=view_citation&hl=en&oe=ASCII&pagesize=100&citation_for_view=rr94cyYAAAAJ:bnK-pcrLprsC
/citations?user=rr94cyYAAAAJ&view_op=view_citation&hl=en&oe=ASCII&pagesize=100&citation_for_view=rr94cyYAAAAJ:RGFaLdJalmkC
/citations?user=

/citations?user=n9DBSCAAAAAJ&view_op=view_citation&hl=en&oe=ASCII&pagesize=100&citation_for_view=n9DBSCAAAAAJ:TFP_iSt0sucC
/citations?user=n9DBSCAAAAAJ&view_op=view_citation&hl=en&oe=ASCII&pagesize=100&citation_for_view=n9DBSCAAAAAJ:BqipwSGYUEgC
/citations?user=n9DBSCAAAAAJ&view_op=view_citation&hl=en&oe=ASCII&pagesize=100&citation_for_view=n9DBSCAAAAAJ:aqlVkmm33-oC
/citations?user=n9DBSCAAAAAJ&view_op=view_citation&hl=en&oe=ASCII&pagesize=100&citation_for_view=n9DBSCAAAAAJ:u9iWguZQMMsC
/citations?user=n9DBSCAAAAAJ&view_op=view_citation&hl=en&oe=ASCII&pagesize=100&citation_for_view=n9DBSCAAAAAJ:nb7KW1ujOQ8C
/citations?user=n9DBSCAAAAAJ&view_op=view_citation&hl=en&oe=ASCII&pagesize=100&citation_for_view=n9DBSCAAAAAJ:fPk4N6BV_jEC
/citations?user=n9DBSCAAAAAJ&view_op=view_citation&hl=en&oe=ASCII&pagesize=100&citation_for_view=n9DBSCAAAAAJ:SP6oXDckpogC
/citations?user=n9DBSCAAAAAJ&view_op=view_citation&hl=en&oe=ASCII&pagesize=100&citation_for_view=n9DBSCAAAAAJ:r0BpntZqJG4C
/citations?user=

/citations?user=B_IufdEAAAAJ&view_op=view_citation&hl=en&oe=ASCII&citation_for_view=B_IufdEAAAAJ:M3NEmzRMIkIC
/citations?user=B_IufdEAAAAJ&view_op=view_citation&hl=en&oe=ASCII&citation_for_view=B_IufdEAAAAJ:vbGhcppDl1QC
/citations?user=B_IufdEAAAAJ&view_op=view_citation&hl=en&oe=ASCII&citation_for_view=B_IufdEAAAAJ:tYavs44e6CUC
/citations?user=B_IufdEAAAAJ&view_op=view_citation&hl=en&oe=ASCII&citation_for_view=B_IufdEAAAAJ:hMsQuOkrut0C
/citations?user=B_IufdEAAAAJ&view_op=view_citation&hl=en&oe=ASCII&citation_for_view=B_IufdEAAAAJ:L7CI7m0gUJcC
/citations?user=B_IufdEAAAAJ&view_op=view_citation&hl=en&oe=ASCII&citation_for_view=B_IufdEAAAAJ:EYYDruWGBe4C
/citations?user=B_IufdEAAAAJ&view_op=view_citation&hl=en&oe=ASCII&citation_for_view=B_IufdEAAAAJ:fEOibwPWpKIC
/citations?user=B_IufdEAAAAJ&view_op=view_citation&hl=en&oe=ASCII&citation_for_view=B_IufdEAAAAJ:JQOojiI6XY0C
/citations?user=B_IufdEAAAAJ&view_op=view_citation&hl=en&oe=ASCII&citation_for_view=B_IufdEAAAAJ:N5tVd3kTz84C
/citations

/citations?user=VVcKUKoAAAAJ&view_op=view_citation&hl=en&oe=ASCII&pagesize=100&citation_for_view=VVcKUKoAAAAJ:u_35RYKgDlwC
/citations?user=VVcKUKoAAAAJ&view_op=view_citation&hl=en&oe=ASCII&pagesize=100&citation_for_view=VVcKUKoAAAAJ:_FxGoFyzp5QC
/citations?user=VVcKUKoAAAAJ&view_op=view_citation&hl=en&oe=ASCII&pagesize=100&citation_for_view=VVcKUKoAAAAJ:Se3iqnhoufwC
/citations?user=VVcKUKoAAAAJ&view_op=view_citation&hl=en&oe=ASCII&pagesize=100&citation_for_view=VVcKUKoAAAAJ:LkGwnXOMwfcC
/citations?user=VVcKUKoAAAAJ&view_op=view_citation&hl=en&oe=ASCII&pagesize=100&citation_for_view=VVcKUKoAAAAJ:D03iK_w7-QYC
/citations?user=VVcKUKoAAAAJ&view_op=view_citation&hl=en&oe=ASCII&pagesize=100&citation_for_view=VVcKUKoAAAAJ:roLk4NBRz8UC
/citations?user=VVcKUKoAAAAJ&view_op=view_citation&hl=en&oe=ASCII&pagesize=100&citation_for_view=VVcKUKoAAAAJ:a0OBvERweLwC
/citations?user=VVcKUKoAAAAJ&view_op=view_citation&hl=en&oe=ASCII&pagesize=100&citation_for_view=VVcKUKoAAAAJ:yD5IFk8b50cC
/citations?user=

/citations?user=J7S_8R8AAAAJ&view_op=view_citation&hl=en&oe=ASCII&pagesize=100&citation_for_view=J7S_8R8AAAAJ:mVmsd5A6BfQC
/citations?user=FKeJ6dwAAAAJ&view_op=view_citation&hl=en&oe=ASCII&pagesize=100&citation_for_view=FKeJ6dwAAAAJ:UeHWp8X0CEIC
/citations?user=FKeJ6dwAAAAJ&view_op=view_citation&hl=en&oe=ASCII&pagesize=100&citation_for_view=FKeJ6dwAAAAJ:u-x6o8ySG0sC
/citations?user=FKeJ6dwAAAAJ&view_op=view_citation&hl=en&oe=ASCII&pagesize=100&citation_for_view=FKeJ6dwAAAAJ:LkGwnXOMwfcC
/citations?user=FKeJ6dwAAAAJ&view_op=view_citation&hl=en&oe=ASCII&pagesize=100&citation_for_view=FKeJ6dwAAAAJ:qjMakFHDy7sC
/citations?user=FKeJ6dwAAAAJ&view_op=view_citation&hl=en&oe=ASCII&pagesize=100&citation_for_view=FKeJ6dwAAAAJ:5nxA0vEk-isC
/citations?user=FKeJ6dwAAAAJ&view_op=view_citation&hl=en&oe=ASCII&pagesize=100&citation_for_view=FKeJ6dwAAAAJ:f2IySw72cVMC
/citations?user=FKeJ6dwAAAAJ&view_op=view_citation&hl=en&oe=ASCII&pagesize=100&citation_for_view=FKeJ6dwAAAAJ:ufrVoPGSRksC
/citations?user=

/citations?user=rHp1m-oAAAAJ&view_op=view_citation&hl=en&oe=ASCII&pagesize=100&citation_for_view=rHp1m-oAAAAJ:_FxGoFyzp5QC
/citations?user=rHp1m-oAAAAJ&view_op=view_citation&hl=en&oe=ASCII&pagesize=100&citation_for_view=rHp1m-oAAAAJ:Se3iqnhoufwC
/citations?user=rHp1m-oAAAAJ&view_op=view_citation&hl=en&oe=ASCII&pagesize=100&citation_for_view=rHp1m-oAAAAJ:hC7cP41nSMkC
/citations?user=rHp1m-oAAAAJ&view_op=view_citation&hl=en&oe=ASCII&pagesize=100&citation_for_view=rHp1m-oAAAAJ:4DMP91E08xMC
/citations?user=rHp1m-oAAAAJ&view_op=view_citation&hl=en&oe=ASCII&pagesize=100&citation_for_view=rHp1m-oAAAAJ:ZeXyd9-uunAC
/citations?user=rHp1m-oAAAAJ&view_op=view_citation&hl=en&oe=ASCII&pagesize=100&citation_for_view=rHp1m-oAAAAJ:_Qo2XoVZTnwC
/citations?user=rHp1m-oAAAAJ&view_op=view_citation&hl=en&oe=ASCII&pagesize=100&citation_for_view=rHp1m-oAAAAJ:aqlVkmm33-oC
/citations?user=rHp1m-oAAAAJ&view_op=view_citation&hl=en&oe=ASCII&pagesize=100&citation_for_view=rHp1m-oAAAAJ:8k81kl-MbHgC
/citations?user=

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [3]:
len(Paper_Links)

34734

### Paper info scraper

In [30]:
def Data_Grabber(Paper_URL_List):
    count = 0

    Dataset_Big = []

    with requests.session() as ses:
        for i in Paper_URL_List:
            auth_topics = ""
            start = 0
            count += 1
        
            while start < 1:
                time.sleep(2.25)
                url = "https://scholar.google.co.uk" + i 
                s = BeautifulSoup(ses.get(url).text, "html.parser")
                x = s.findAll("div", attrs = {"class": "gs_scl"})
            
                title = s.find("div", attrs = {"id": "gsc_vcd_title"}).text
            
                start += 1
            
                authors = ""
                date = ""
                snippet = ""
            
                for t in x:
                    if t.find("div", attrs = {"class": "gsc_vcd_field"}).text == "Authors":
                        authors = t.find("div", attrs = {"class": "gsc_vcd_value"}).text
                
                    if t.find("div", attrs = {"class": "gsc_vcd_field"}).text == "Publication date":
                        date = t.find("div", attrs = {"class": "gsc_vcd_value"}).text
                
                    if t.find("div", attrs = {"class": "gsc_vcd_field"}).text == "Description":
                        snippet = t.find("div", attrs = {"class": "gsc_vcd_value"}).text
                    
                total_paper_data = {"Authors": authors, "Paper": title, "Publish Date": date, "Description": snippet}
                
                Dataset_Big.append(total_paper_data)
                
            with open("Dataset_Big.csv", mode = "w", newline = "", encoding="utf-8") as Data_Big:
                Data_Writer = csv.DictWriter(Data_Big, fieldnames = ["Authors", "Paper", "Publish Date", "Description"])
                Data_Writer.writeheader()
                for i in Dataset_Big:
                    Data_Writer.writerow(i)
            
            print(count, title)
        return Dataset_Big

In [ ]:
Data_Grabber(Paper_Links)

1 Still Want to Know Who Is the Human Response
2 18 Autonomous Mobile Robot
3 Distinguishing AI from Male/Female Dialogue.
4 [BOOK] Turing's Imitation Game
5 Distinguished keynote speaker
6 Distinguished keynote speaker: The Cyborg Experiments
7 ICITST honorary chair and distinguished keynote speaker: The cyborg experiments
8 Interview [Interview Kevin Warwick]
9 The Debate: TV Robots
10 Resting tremor classification and detection in Parkinson's disease patients
11 [CITATION] Intelligenza artificiale. Le basi
12 Can machines think
13 Keynote I.
14 Authors' Reply to'Comments on" liveness of an extended S 3PR"[Automatica 46 (6)(2010) 1008-1018]'
15 Fundamental Artificial Intelligence
16 [CITATION] Fundamental Artificial Intelligence: Machine Performance in Turing's Two Tests for Thinking.{March, Angers: France}
17 [CITATION] Fundamental Artificial Intelligence: Machine Performance in Turing's Two Tests for Thinking.{March, Angers: France}
18 D6. 2-Guidelines on regulating robotics Pisa
1